In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm, chi2
from joblib import Parallel, delayed
from tqdm import tqdm

# -----------------------------
# ODE model
# -----------------------------
def ode_rhs(t, y, p):
    a1, a2, a3, a4, a5, a6 = p
    y1, y2 = y
    dy1 = a1 * y1 * (1 - y1 / a2) - a3 * y1 * y2
    dy2 = a4 - a5 * y2 - a6 * y1 * y2
    return [dy1, dy2]

def solve_dataset(t_obs, params, y10, y20, method="BDF", rtol=1e-5, atol=1e-7):
    sol = solve_ivp(lambda t, y: ode_rhs(t, y, params),
                    (t_obs[0], t_obs[-1]), [y10, y20],
                    t_eval=t_obs, method=method, rtol=rtol, atol=atol)
    return sol.y[0] + sol.y[1]

# -----------------------------
# Experimental setup
# -----------------------------
t_obs = [
    np.array([14, 16, 18, 20, 22, 24, 26, 28, 30], float),
    np.array([12, 14, 16, 18, 20, 22, 24, 26], float),
    np.array([ 6,  8, 10, 12, 14, 16, 18, 20], float),
    np.array([ 2,  4,  6,  8, 10, 12, 14], float)
]


theta_true = np.array([
   0.39562, 890447238, 1.06870e-07,
    75198,   0.00122, 1.59893e-09,
    41905,
     9381768, 24694962, 23810412, 61908988
])

lb_kin = np.array([0.35, 1.6e9, 0.8e-8, 8e4, 1e-3, 1e-9], float)
ub_kin = np.array([0.38, 2.2e9, 2.7e-8, 1.4e5, 1.8e-1, 5.5e-9], float)
y1min, y1max = 1e3, 5e8
y2min, y2max = 1e1, 1e7
lb_all = np.concatenate([lb_kin, [y2min], [y1min]*4])
ub_all = np.concatenate([ub_kin, [y2max], [y1max]*4])
bounds_list = list(zip(lb_all, ub_all))

# -----------------------------
# Generate noisy data
# -----------------------------
CV, SigmaFloor = 0.025, 1e5
exp_data, sigma_data = [], []
params_true = theta_true[:6]
y2_true = theta_true[6]
y1_trues = theta_true[7:]

for i, y10 in enumerate(y1_trues):
    clean = solve_dataset(t_obs[i], params_true, y10, y2_true, "BDF")
    sigma_vals = np.maximum(CV * clean, SigmaFloor)
    noisy = clean + np.random.normal(0, sigma_vals)
    exp_data.append(noisy)
    sigma_data.append(sigma_vals)

print("Synthetic noisy data generated.")

# -----------------------------
# Log-likelihood
# -----------------------------
def loglikelihood(theta, method="BDF"):
    a_params = theta[:6]
    y20 = theta[6]
    y10s = theta[7:]
    ll_total = 0.0
    for i in range(4):
        pred = solve_dataset(t_obs[i], a_params, y10s[i], y20, method, rtol=1e-6, atol=1e-8)
        resid = exp_data[i] - pred
        sigmas = sigma_data[i]
        ll_total += np.sum(norm.logpdf(resid, loc=0, scale=sigmas))
    return ll_total

def neg_ll(theta):
    return -loglikelihood(theta, method="BDF")

# -----------------------------
# Compute MLE with global + local search
# -----------------------------
print("Running global optimization (this may take time)...")
res_global = differential_evolution(neg_ll, bounds_list, maxiter=50, polish=False, workers=-1)
print("Global search done. Refining with local search...")
res_local = minimize(neg_ll, res_global.x, bounds=bounds_list,
                     method="L-BFGS-B", options={"maxiter":500, "ftol":1e-8})
theta_mle = res_local.x
print("Optimization success:", res_local.success)
print("MLE parameters:", theta_mle)

# -----------------------------
# Profile likelihood with warm starts
# -----------------------------
def profile_likelihood(param_index, theta_mle, bounds, method="BDF", n_grid=20):
    p_grid = np.linspace(bounds[param_index][0], bounds[param_index][1], n_grid)
    ll_values = np.empty_like(p_grid)
    ll_mle = loglikelihood(theta_mle, method)

    # Initial guess for nuisance = from MLE
    nuisance_guess = [theta_mle[i] for i in range(len(theta_mle)) if i != param_index]

    for j, val in enumerate(tqdm(p_grid, desc=f"Param {param_index+1}")):
        theta_fixed = theta_mle.copy()
        theta_fixed[param_index] = val
        free_idx = [i for i in range(len(theta_mle)) if i != param_index]
        free_bounds = [bounds[i] for i in free_idx]

        def neg_ll_nuisance(free_params):
            theta_var = theta_fixed.copy()
            theta_var[free_idx] = free_params
            return -loglikelihood(theta_var, method)

        res = minimize(neg_ll_nuisance, nuisance_guess, bounds=free_bounds,
                       method="L-BFGS-B", options={"ftol":1e-8, "gtol":1e-8, "maxiter":200})
        ll_values[j] = -res.fun
        nuisance_guess = res.x  # warm start for next step

    norm_ll = ll_values - ll_mle
    return param_index, p_grid, norm_ll

# -----------------------------
# Run all profiles in parallel
# -----------------------------
results = Parallel(n_jobs=8)(
    delayed(profile_likelihood)(i, theta_mle, bounds_list, "BDF", n_grid=15)
    for i in range(6)
)

# -----------------------------
# Plot all results
# -----------------------------
param_labels = [r"$c_1$", r"$c_{\max}$", r"$\phi_e$", r"$\gamma_e$", r"$\delta_e$", r"$\eta_e$"]

fig, axes = plt.subplots(2, 3, figsize=(15,8))
axes = axes.ravel()
cutoff = -chi2.ppf(0.95, df=1)/2

for param_index, p_grid, norm_ll in results:
    ax = axes[param_index]
    ax.plot(p_grid, norm_ll, color='blue', lw=2, label="Profile")
    ax.plot(p_grid, norm_ll, 'ro', markeredgecolor='black', label="Steps")
    ax.axhline(cutoff, color='green', linestyle='--', label='95% CI cutoff')
    ax.axvline(theta_mle[param_index], color='black', linestyle=':', label='MLE')
    ax.set_xlabel(param_labels[param_index])
    ax.set_ylabel(r"$\Delta$ log-likelihood")
    ax.legend()

plt.tight_layout()
plt.show()
